In [1]:
from delta.tables import *
#from notebookutils import mssparkutils
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
#from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import col, when, from_json, date_format, lit, row_number,max, lower, regexp_extract
from pyspark.sql.types import StructType, StringType
from pyspark.sql.window import Window

StatementMeta(, 0c5a145a-7611-4570-b990-4dd58dee0e58, 3, Finished, Available, Finished)

In [2]:
fromMonth = -2 #-1, -2,... from datenow -1 day
toMonth = -1 #-1, -2,... from datenow -1 day

StatementMeta(, 0c5a145a-7611-4570-b990-4dd58dee0e58, 4, Finished, Available, Finished)

## FUNCTIONS

In [3]:
def generateArrayOFPeriod(from_Month: int, to_month: int):
    # Get today's date
    today = datetime.today()

    # Subtract one day
    yesterday = today - timedelta(days=1)
    first_day = yesterday.replace(day=1)

    periodToLoad = []
    if from_Month == to_month:
        periodDate = first_day + relativedelta(months=to_month)
        periodToLoad.append(periodDate.date())
    else:
        for i in range(from_Month, to_month+1):
            periodDate = first_day + relativedelta(months=i)
            periodToLoad.append(periodDate.date())

    return periodToLoad

StatementMeta(, 0c5a145a-7611-4570-b990-4dd58dee0e58, 5, Finished, Available, Finished)

## STEP 2 Load Gold:

- Filter on Fabric data only
- Remove x_ column
- Add FabricPause column

In [4]:
periodsToLoad = generateArrayOFPeriod(fromMonth, toMonth)

StatementMeta(, 0c5a145a-7611-4570-b990-4dd58dee0e58, 6, Finished, Available, Finished)

In [5]:
# Preparation of predicate for all ingested periods
date_condition = ", ".join([f"'{date.strftime('%Y-%m-%d')}'" for date in periodsToLoad])

print("Start loading Period(s) in " + date_condition)

#clean existing data in silver
if spark._jsparkSession.catalog().tableExists( "reservation_usage"):
    print("Table exists, snapshot will be clean.")
    Delete_sql_query = f"""DELETE FROM reservation_usage WHERE PeriodLoaded IN ({date_condition})"""
    spark.sql(Delete_sql_query)
    print("Clean performed")

reservation_usage_df = DeltaTable.forPath(spark,"Tables/reservation_usage_silver").toDF()
reservation_usage_df = reservation_usage_df.where(f"""PeriodLoaded IN ({date_condition})""")

StatementMeta(, 0c5a145a-7611-4570-b990-4dd58dee0e58, 7, Finished, Available, Finished)

Start loading Period(s) in '2025-08-01', '2025-09-01'


In [6]:
#Filter on Fabric RI usage only
reservation_usage_df=reservation_usage_df.filter(lower(col("InstanceId")).contains("providers/microsoft.fabric/"))\
                                        .withColumn("SubAccountId",regexp_extract(lower(col("InstanceId")), r"subscriptions\/(.*?)\/", 1) )\
                                        .withColumn("ResourceGroupName",regexp_extract(lower(col("InstanceId")), r"resourcegroups\/(.*?)\/", 1) )\
                                        .withColumn("ResourceName",regexp_extract(lower(col("InstanceId")), r"capacities\/(.*?)$", 1) )\
                                        .drop("InstanceId")

reservation_df = DeltaTable.forPath(spark,"Tables/reservations").toDF()
reservation_df = reservation_df.select("ReservationOrderId","ReservationOrderKey")
reservation_usage_df = reservation_usage_df.join(reservation_df,"ReservationOrderId", "leftouter") #retrieve ReservationOrderKey

subscription_df = DeltaTable.forPath(spark,"Tables/subscriptions").toDF()
subscription_df = subscription_df.select("SubAccountId","SubscriptionKey")
reservation_usage_df = reservation_usage_df.join(subscription_df,"SubAccountId", "leftouter") #retrieve SubscriptionKey

resources_df = DeltaTable.forPath(spark,"Tables/resources").toDF()
resources_df = resources_df.select("ResourceName","ResourceKey")
reservation_usage_df = reservation_usage_df.join(resources_df,"ResourceName", "leftouter") #retrieve ResourceKey

reservation_usage_df.write.mode("append").option("mergeSchema", "true").format("delta").saveAsTable("reservation_usage")

print("End Loading")

StatementMeta(, 0c5a145a-7611-4570-b990-4dd58dee0e58, 8, Finished, Available, Finished)

End Loading
